In [3]:
from utils import *
from utils.sql_functions import *

def write_list_to_file(myList, output_path, var_name):
    with open(output_path, 'w') as f:
        f.write("{}='''".format(var_name))
        f.write('\n'.join([str(x) for x in myList]))
        f.write("'''.split('\\n')")

# Data Preparation

## Context Data

- User Context

In [702]:
def prepare_user_context():
    with closing(get_connection()) as conn: # ensure that the connection is closed        
        statement = '''SELECT userId AS userID, height, weight, gender
                        FROM user_info'''
        user_data = query(conn, statement)
    with closing(get_connection()) as conn: # ensure that the connection is closed
        statement = '''SELECT userId AS userID, startTime, exerciseClassId, effectTime, age
                        FROM user_exercise 
                        WHERE isMeetLeastEffectTime = "Y" '''
        df = query(conn, statement)

    df['time_window'] = [str(x)[:7] for x in df['startTime']]
    del df['startTime']
    df = df.drop_duplicates()
    
    # Group by 'userID', 'time_window', 'age'
    dfs = [df.groupby(['userID', 'time_window', 'age']).agg({'effectTime' : ['mean', 'max', 'min', 'std', 'count']}).reset_index()]
    dfs[0].columns = ['userID', 'time_window', 'age', 
                      'all_effectTime_mean', 'all_effectTime_max', 'all_effectTime_min', 'all_effectTime_std', 'all_count']
    
    # Group by 'userID', 'time_window', 'age', 'exerciseClassId'
    df = df.groupby(['userID', 'time_window', 'age', 'exerciseClassId']).agg({'effectTime' : ['mean', 'max', 'min', 'std', 'count']}).reset_index()
    df.columns = ['userID', 'time_window', 'age', 'exerciseClassId',
                  'effectTime_mean', 'effectTime_max', 'effectTime_min', 'effectTime_std', 'count']
    exerciseClassIdList = df['exerciseClassId'].unique()
    dfs += [df[df['exerciseClassId'] == i].rename(columns={"effectTime_mean": "effectTime_mean_{}".format(i), 
                                                           "effectTime_max": "effectTime_max_{}".format(i), 
                                                           "effectTime_min": "effectTime_min_{}".format(i), 
                                                           "effectTime_std": "effectTime_std_{}".format(i), 
                                                           "count": "count_{}".format(i)}) \
                                          .drop(columns=['exerciseClassId']) for i in exerciseClassIdList]
    
    dfs = [df.set_index(['userID', 'time_window', 'age']) for df in dfs]
    df = dfs[0].join(dfs[1:]).reset_index()
    user_context = df.merge(user_data, on='userID', how='inner')
    #print(user_context.isnull().sum())
    
    user_context = user_context.fillna(0)
    return user_context

- RestHR Context

In [704]:
def prepare_restHR_context():
    with closing(get_connection()) as conn: #ensure that the connection is closed
        statement = "SELECT userId, restHRMeasureWay, createdTime, restHR FROM l_user_restHR"
        user_restHR = query(conn, statement)
    user_restHR['createdMonth'] = [str(x)[:7] for x in user_restHR['createdTime']]
    user_restHR[['userId', 'restHR']].groupby('userId').count().reset_index()\
                                     .rename(columns={'restHR': 'count'})
    user_restHR[['userId', 'createdMonth', 'restHR']].groupby(['userId', 'createdMonth']).count().reset_index()\
                                                     .rename(columns={'restHR': 'count'})
    restHR_context = user_restHR[['userId', 'createdMonth', 'restHR']].groupby(['userId', 'createdMonth'])\
                                                                      .agg({'restHR': ['mean', 'count']}).reset_index()
    restHR_context.columns = ['userID', 'time_window', 'restHR_mean', 'restHR_count']
    restHR_context = restHR_context.drop_duplicates()
    return restHR_context

In [706]:
user_context = prepare_user_context()
restHR_context = prepare_restHR_context()

## Combine & Labeling

In [777]:
def prepare_data(user_context, restHR_context, exerciseHR_context=None):
    with closing(get_connection()) as conn: # ensure that the connection is closed        
        statement = '''SELECT userId AS userID, birthday, gender, height, weight
                        FROM user_info'''
        _ = query(conn, statement)
        user_dict = _.set_index(['userID']).to_dict('index')
    
    def calculate_age(user_dict, userID, end_date):
        start_date = get_dict_value(user_dict, userID, 'birthday')
        if type(end_date) == str:
            end_date = datetime.strptime(end_date, "%Y-%m").date()
        return relativedelta(end_date, start_date).years

    def get_dict_value(user_dict, key, col, fill_val=None):
        try:
            return user_dict[key][col]
        except KeyError:
            return fill_val
    
    def fill_duration_context(data, context_dic, columns, duration_range, fill_val=0):
        for col in columns:
            for i in duration_range:
                data['{}_m{}'.format(col, 1+i)] = [
                    get_dict_value(context_dic, (user, str(m + relativedelta(months=i))[:7]), col, fill_val) \
                    for user, m in zip(data['userID'], data['time_window'])
                ]
        return data
    
    valid_HR = {
        0:{
            '(17, 25]':78, '(25, 35]':76, '(35, 45]':78, 
            '(45, 55]':77, '(55, 65]':77, '(65, 100]':76,
        },
        1:{
            '(17, 25]':73, '(25, 35]':74, '(35, 45]':75,
            '(45, 55]':76, '(55, 65]':75, '(65, 100]':73,
        }
    }
    restHR_data = restHR_context.copy()
    print("Original", len(restHR_data['userID'].unique()))
    
    # Add user info
    restHR_data['age'] = [calculate_age(user_dict, userID, end_date) for userID, end_date in \
                          zip(restHR_data['userID'], restHR_data['time_window'])]
    for col in ['height', 'weight', 'gender']:
        restHR_data[col] = [get_dict_value(user_dict, userID, col) for userID in restHR_data['userID']]
    restHR_data = restHR_data.dropna()
    print("Add user info", len(restHR_data['userID'].unique()))
    
    # Age binning
    age_intervals = [17, 25, 35, 45, 55, 65, 100]
    restHR_data['AGE'] = pd.cut(restHR_data['age'], bins=age_intervals)
    restHR_data['time_window'] = [datetime.strptime(x, '%Y-%m') for x in restHR_data['time_window']]
    restHR_data = restHR_data.dropna()
    print("Age binning", len(restHR_data['userID'].unique()))

    # Labeling - m1
    restHR_dict = restHR_context.set_index(['userID', 'time_window']).to_dict('index')
    restHR_data['belowAvg'] = [
        hr > valid_HR[gender][str(age)] for hr,gender,age in zip(restHR_data['restHR_mean'], restHR_data['gender'], restHR_data['AGE'])
    ]
    restHR_data = restHR_data.drop_duplicates(['userID', 'time_window'])
    restHR_data['restHR_count'] = [
        get_dict_value(restHR_dict, (user, str(m)[:7]), 'restHR_count') \
        for user, m in zip(restHR_data['userID'], restHR_data['time_window'])
    ]
    print("Labeling - m1", len(restHR_data[restHR_data['belowAvg'] == True]['userID'].unique()))
    
    # Labeling - m6
    restHR_data['target_restHR_mean'] = [
        get_dict_value(restHR_dict, (user, str(m + relativedelta(months=5))[:7]), 'restHR_mean') \
        for user, m in zip(restHR_data['userID'], restHR_data['time_window'])
    ]
    restHR_data['target_restHR_count'] = [
        get_dict_value(restHR_dict, (user, str(m + relativedelta(months=5))[:7]), 'restHR_count') \
        for user, m in zip(restHR_data['userID'], restHR_data['time_window'])
    ]
    
    # Select Users
    data = restHR_data[restHR_data['belowAvg'] == True]
    data = data[data['target_restHR_mean'].isnull() == False]
    data['target_belowAvg'] = [
        hr > valid_HR[gender][str(age)] for hr,gender,age in \
        zip(data['target_restHR_mean'], data['gender'], data['AGE'])
    ]
    print("Labeling - m6", len(data['userID'].unique()))
    
    # Add exercise records
    user_dict = user_context.set_index(['userID', 'time_window']).to_dict('index')
    columns = list(set(user_context.columns) - set(['userID', 'time_window', 'age', 'height', 'weight', 'gender']))
    data = fill_duration_context(data, user_dict, columns, range(1, 5), fill_val=0)
    data['total_count'] = data[list(filter(lambda x: x.startswith('count_'), data.columns))].sum(axis=1)
    data = data[data['total_count'] > 0]
    
    # Some statistics 
    for i in range(3):
        print("At least {} exercise".format(i+1), len(data[data['total_count'] > i]['userID'].unique()))
    tmp  = data.copy()
    for i in range(1, 5):
        tmp = tmp[tmp['all_count_m{}'.format(i+1)] > 0]
    print("At least 1 exercise per month", len(tmp['userID'].unique()))
    
    # Add exerciseHR records
    if exerciseHR_context is not None:
        exerciseHR_dict = exerciseHR_context.set_index(['userID', 'time_window']).to_dict('index')
        columns = list(set(exerciseHR_context.columns) - set(['userID', 'time_window']))
        data = fill_duration_context(data, exerciseHR_dict, columns, range(1, 5), fill_val=None)
    return data

In [778]:
data = prepare_data(user_context, restHR_context, exerciseHR_context=None)

('Original', 7999)
('Add user info', 7997)
('Age binning', 7916)
('Labeling - m1', 3404)
('Labeling - m6', 289)
('At least 1 exercise', 245)
('At least 2 exercise', 235)
('At least 3 exercise', 229)
('At least 1 exercise per month', 155)


In [779]:
data.isnull().sum()

userID                   0
time_window              0
restHR_mean              0
restHR_count             0
age                      0
height                   0
weight                   0
gender                   0
AGE                      0
belowAvg                 0
target_restHR_mean       0
target_restHR_count      0
target_belowAvg          0
all_effectTime_min_m2    0
all_effectTime_min_m3    0
all_effectTime_min_m4    0
all_effectTime_min_m5    0
effectTime_std_26_m2     0
effectTime_std_26_m3     0
effectTime_std_26_m4     0
effectTime_std_26_m5     0
effectTime_min_9_m2      0
effectTime_min_9_m3      0
effectTime_min_9_m4      0
effectTime_min_9_m5      0
all_effectTime_std_m2    0
all_effectTime_std_m3    0
all_effectTime_std_m4    0
all_effectTime_std_m5    0
effectTime_min_13_m2     0
                        ..
effectTime_mean_12_m5    0
effectTime_mean_13_m2    0
effectTime_mean_13_m3    0
effectTime_mean_13_m4    0
effectTime_mean_13_m5    0
effectTime_mean_10_m2    0
e

In [780]:
len(data)

352

In [781]:
data.to_csv('info/data.health_state_prediction_task.csv', index=None)

# Data Selection

In [4]:
data = pd.read_csv('info/data.health_state_prediction_task.csv', index_col=False, parse_dates=['time_window'])

In [5]:
data['avg_count'] = [x/4 for x in data['total_count']]
data['avg_count'].describe()

count    352.000000
mean      13.977273
std       25.938431
min        0.250000
25%        2.750000
50%        7.750000
75%       17.312500
max      362.750000
Name: avg_count, dtype: float64

In [6]:
users_2015 = pd.read_csv('info/active_users.2015.csv')['userId'].unique()
users_2016 = pd.read_csv('info/active_users.2016.csv')['userId'].unique()
users_2017 = pd.read_csv('info/active_users.2017.csv')['userId'].unique()
users_2018 = pd.read_csv('info/active_users.2018.csv')['userId'].unique()

In [7]:
valid_data = [data[(data['time_window'] >= '2015-01-01') & (data['time_window'] < '2016-01-01')], 
              data[(data['time_window'] >= '2016-01-01') & (data['time_window'] < '2017-01-01')], 
              data[(data['time_window'] >= '2017-01-01') & (data['time_window'] < '2018-01-01')], 
              data[(data['time_window'] >= '2018-01-01') & (data['time_window'] < '2019-01-01')] ]

In [8]:
valid_data[0] = valid_data[0][valid_data[0]['userID'].isin(users_2015)]
valid_data[1] = valid_data[1][valid_data[1]['userID'].isin(users_2016)]
valid_data[2] = valid_data[2][valid_data[2]['userID'].isin(users_2017)]
valid_data[3] = valid_data[3][valid_data[3]['userID'].isin(users_2018)]

In [26]:
def get_sample_info(data):
    data_dict = data[['userID', 'time_window']].to_dict('records')
    for sample in data_dict:
        m = sample['time_window']
        sample['exercise_months'] = [str(m + relativedelta(months=i))[:7] for i in range(1, 5)]
        del sample['time_window']
    select_context = {}
    for sample in data_dict:
        if sample['userID'] in select_context:
            select_context[sample['userID']] += sample['exercise_months']
        else:
            select_context[sample['userID']] = sample['exercise_months']
    return select_context

## Save Train Set

In [36]:
data = pd.concat(valid_data[:2])
data.to_csv('health_state_prediction/data/data.train.csv', index=None)

In [37]:
select_context = get_sample_info(data)
with open('info/samples.health_state_prediction.train.json', 'w') as f:
    json.dump(select_context, f)

## Save Test Set

In [38]:
data = pd.concat(valid_data[2:])
data.to_csv('health_state_prediction/data/data.test.csv', index=None)

In [39]:
select_context = get_sample_info(data)
with open('info/samples.health_state_prediction.test.json', 'w') as f:
    json.dump(select_context, f)